In [ ]:
from operator import truediv
import cv2
import numpy as np
import matplotlib.pyplot as plt
from ast import While
from lib2to3.pytree import WildcardPattern
from tokenize import Triple
vedio = cv2.VideoCapture("solidWhiteRight.mp4")
i=0
cv2.namedWindow("image")

def canny(img):
  low_th=40
  high_th=100
  blur = cv2.GaussianBlur(img, (7,7), 0)
  return cv2.Canny(blur, low_th, high_th)
def maskROI(img):
  mask = np.zeros_like(img)
  if len(img.shape) > 2:
    channel_count = img.shape[2]
    ignore_mask_color = (255,) * channel_count
  else:
    ignore_mask_color = 255
  imshape = img.shape        
  vertices = np.array([
    [
      ((1/6*imshape[1]), imshape[0]),
      ((5/12*imshape[1]), (3/5*imshape[0])),
      ((7/12*imshape[1]), (3/5*imshape[0])),
      ((9/10*imshape[1]), imshape[0])
    ]], dtype=np.int32)
  cv2.fillPoly(mask, vertices, ignore_mask_color)
  masked_edges = cv2.bitwise_and(img, mask)
  return(masked_edges)
def hough_lines(img2):
  lines = cv2.HoughLinesP(img2, RHO, THETA, MIN_VOTES, np.array([]), minLineLength=MIN_LINE_LEN, maxLineGap=MAX_LINE_GAP)
  return lines
def draw_lines(lines, masked_edges):
  color = [243, 105, 14]
  thickness = 12
  lines_image = np.zeros((masked_edges.shape[0], masked_edges.shape[1], 3), dtype=np.uint8)
  for line in lines:
    for x1,y1,x2,y2 in line:
      cv2.line(lines_image, (x1, y1), (x2, y2), color, thickness)
  return(lines_image) 


while vedio.isOpened():
  stat, img=vedio.read()
  if stat ==True:          
    k=cv2.waitKey(1)
    if k==ord('q'):
      break
    if k==ord('s'):
      cv2.imwrite('img'+str(i)+'.png',img)
      i+=1
    edges1 = canny(img)
    masked = maskROI(edges1)
    test = np.ones((500, 1000))*100
    test[-1, 500] = 255

    RHO = 1                 # try: 1 - 4 (0.5 increments)  
    THETA = np.pi/180       # Usually this is Ok
    MIN_VOTES = 10          # try: 10 - 50                  # Typical: 30
    MIN_LINE_LEN = 5 
    MAX_LINE_GAP = 50 
  
    lines = hough_lines(masked)
    lined_img = draw_lines(lines, masked)
    from helpers import helpers_formulate_lanes as formulate_lanes
    lanes = formulate_lanes(lines, masked)
    lanes_img = draw_lines(lanes, img)
    final_image = cv2.addWeighted(img, 0.9, lanes_img, 1, 0)
    cv2.imshow("image",final_image)
    
  
             
  

  else:
    k=cv2.waitKey(1)
    if k==ord('q'):
      break
    cv2.destroyAllWindows()   
    print("wrong frame") 

    break     




vedio.release()
cv2.destroyAllWindows() 